In [ ]:
import pandas as pd 
import random
from pyspark.sql import SparkSession

StatementMeta(, , , Waiting, )

In [ ]:
def update_value(value):
    if value>=2:
        return 'High'
    elif value==2:
        return 'Medium'
    else:
        return 'Low'

StatementMeta(, , , Waiting, )

In [ ]:
def add_external_values(df_write):
    df = spark.read.format("delta").load("Tables/loan_master")  
    df = df.toPandas()
    df = df.drop_duplicates(subset='cust_id', keep="first")
    open_prob=0.7
   # Add new columns only if they don't exist
    if 'CRILC_Defaulter_List' not in df.columns:
        df['CRILC_Defaulter_List'] = ['yes' if random.random() < open_prob else 'No' for _ in range(len(df))]
    
    if 'Wilful_Defaulter_List' not in df.columns:
        df['Wilful_Defaulter_List'] = ['yes' if random.random() < open_prob else 'No' for _ in range(len(df))]
    

    df['ratings_2023']=df['ratings_2023'].replace({'AAA':7,'AA':6,'A':5,
                                                       'BBB':4,'BB':3,'B':2,
                                                       'C':1,'D':0})
    df['ratings_2022']=df['ratings_2022'].replace({'AAA':7,'AA':6,'A':5,
                                                       'BBB':4,'BB':3,'B':2,
                                                       'C':1,'D':0})
    df['External_Rating_Alert']=df['ratings_2023']-df['ratings_2022']
    df['External_Rating_Alert']=df['External_Rating_Alert'].apply(update_value)
#     return df

# result_df = add_external_values(df_write)
# display(result_df)

    names_values={
        'Regulatory_News_Alert':27,
        'Statutory_Dues_Delay':28,
        'Vendor_Loss_Alert':29,
        'Penalty_Alert':30,
        'Business_Expansion_Alert':31,
        'Competition_Alert':32,
        'Low_Market_Demand':33,
        'Employee_Attrittion':34,
        'Exchange_Filing_Alert':35,
        'Cross_Border_Alert':36,
        'Value_Chain_Alert':37,
        'Personal_Loans_Promoters':38,
        'CRILC_Defaulter_List':25,
        'Wilful_Defaulter_List':26,
        'External_Rating_Alert':24
                
             }

    for i in list(names_values.keys()):
        if names_values[i] not in [25,26]:
            df_high = df[['cust_id',i,'Renewal_Due_Date']].copy()
            df_high[i]=df[i].replace({'High':3,'Low':1,'Medium':2})
        else:
            df_high = df[['cust_id',i,'Renewal_Due_Date']].copy()
            df_high=df_high[df_high[i]=='yes'][['cust_id',i,'Renewal_Due_Date']]

        df_high.rename(columns = {i:'alert_impact'}, inplace = True)
        df_high['alert_impact'] = df_high['alert_impact'].map({'High':3,'Medium':2,'Low':1})
        df_high['alert_type']=5
        df_high['alert_attribute']=names_values[i]
        df_high['alert_date']=pd.to_datetime(df_high['Renewal_Due_Date'],format='mixed')+pd.DateOffset(days=random.choice([1, 4, 8, 10, 3,5,7,8]))
        df_high['alert_tat']= [random.randint(1,60) for _ in range(len(df_high))]
        df_high['Date_action_taken']=pd.to_datetime(df_high['alert_date'])+pd.to_timedelta(df_high['alert_tat'],unit='D' )
        open_prob=0.7
        df_high['alert_status']=['open' if random.random()<open_prob else 'Closed' for _ in range(len(df_high))]
        df_high=df_high.drop(['Renewal_Due_Date'], axis=1)
        df_high = df_high.drop_duplicates()
        df_write = pd.concat([df_write, df_high])
    return df_write

StatementMeta(, , , Waiting, )

In [ ]:
def add_internal_values(df_write):
    df_in = spark.read.format("delta").load("Tables/internalfile")  
    df_in = df_in.toPandas()
    df_in = df_in.drop_duplicates(subset='cust_id', keep="first")
    df_loan = spark.read.format("delta").load("Tables/loan_master")  
    df_loan = df_loan.toPandas()
    df_loan = df_loan.drop_duplicates(subset='cust_id', keep="first")
    #merge both the dataframes
    df = df_in.merge(df_loan, on="cust_id", how = 'inner')
    names_values={
        'Auditor_Report_Discrepancies':16,
        'Internal_Fraud_Alert':17,
        'Internal_Dispute':18,
        'Change_in_Statutory_Auditors':19,
        'Resignation_Key_Personnel':20,
        'Project_Sponsor_Withdrawal':21,
        'Financial_Results_Announcement':22,
        'Utility_Disruption':23
                
             }

    for i in list(names_values.keys()):
        df_high = df[['cust_id',i,'Renewal_Due_Date']].copy()
        df_high.rename(columns = {i:'alert_impact'}, inplace = True)
        df_high['alert_impact'] = df_high['alert_impact'].map({'High':3,'Medium':2,'Low':1})

        
        df_high['alert_type']=4
        df_high['alert_attribute']=names_values[i]
        df_high['alert_date']=pd.to_datetime(df_high['Renewal_Due_Date'],format='mixed')+pd.DateOffset(days=random.choice([1, 4, 8, 10, 3,5,7,8]))
        df_high['alert_tat']= [random.randint(1,60) for _ in range(len(df_high))]
        df_high['Date_action_taken']=pd.to_datetime(df_high['alert_date'])+pd.to_timedelta(df_high['alert_tat'],unit='D' )
        open_prob=0.7
        df_high['alert_status']=['open' if random.random()<open_prob else 'Closed' for _ in range(len(df_high))]
        df_high['alert_assgined_to']= [random.randint(1,20) for _ in range(len(df_high))]
        df_high['alert_resolved_by']= [random.randint(1,20) for _ in range(len(df_high))]
        df_high['account_type']=0
        df_high['industry_type']= 0
        df_high=df_high.drop(['Renewal_Due_Date'], axis=1)
        df_high = df_high.drop_duplicates()
        df_write = pd.concat([df_write, df_high])
    return df_write

StatementMeta(, , , Waiting, )

In [ ]:
operationa_dict={
                'Renewal_Due_Date_Days':{
                                    'high':15,'med':[15,5],'low':[5,1],'attr':1
                                    },
                'COD_delay':{
                                    'high':90,'med':[90,30],'low':[30,1],'attr':2
                                    },
                'Security_Perfection_Alert':{
                                    'high':30,'med':[30,20],'low':[20,15],'attr':3
                                    },
                'Closure_Pending_Internal_Audit':{
                                    'high':60,'med':[60,30],'low':[30,0],'attr':4
                                    },
                'Closure_Pending_External_Audit':{
                                    'high':20,'med':[20,15],'low':[15,0],'attr':5
                                    },
                'Closure_Pending_Stock_Audit':{
                                    'high':20,'med':[20,15],'low':[15,0],'attr':6
                                    },
                'Closure_Pending_Revenue_Audit':{
                                    'high':20,'med':[20,15],'low':[15,0],'attr':7
                                    },
                'Delay_SS_Submission':{
                                    'high':10,'med':[10,5],'low':[5,1],'attr':8
                                    },
                'ROC_Filing_Date':{
                                    'high':20,'med':[20,15],'low':[15,0],'attr':10
                                    },
                'CERSAI_Registration_Date':{
                                    'high':20,'med':[20,15],'low':[15,0],'attr':11
                                    },
                'Release_Deed_Date':{
                                    'high':20,'med':[20,15],'low':[15,0],'attr':12
                                    },
                }

In [ ]:
# Read operational data

df_op = spark.read.format("delta").load("Tables/operational")  
df_op = df_op.toPandas()
df_op = df_op.drop_duplicates(subset='cust_id', keep="first")

df_loan = spark.read.format("delta").load("Tables/loan_master")  
df_loan = df_loan.toPandas()
df_loan = df_loan.drop_duplicates(subset='cust_id', keep="first")

# Merge both dataframes
df = df_op.merge(df_loan, on="cust_id", how='inner')

# Create an empty dataframe for alerts
df_write = pd.DataFrame(columns=['cust_id', 'alert_impact', 'alert_type', 'alert_attribute', 'alert_date',
                                 'alert_tat', 'Date_action_taken', 'alert_status', 'alert_assgined_to',
                                 'alert_resolved_by', 'account_type', 'industry_type'])



# Iterate over each key in 'operational_dict'
for i in list(operational_dict.keys()):
    # High alert
    df[i] = pd.to_numeric(df[i],errors='coerce')
    filtered_values_high = df[df[i] > operationa_dict[i]['high']]
    # print(f"Columns of filtered_values_high: {filtered_values_high.columns}")
    df_high = filtered_values_high[['cust_id', 'Renewal_Due_Date']].copy()
    df_high['alert_impact'] = 3
    df_high['alert_type'] = 1
    df_high['alert_attribute'] = operationa_dict[i]['attr']
    df_high['alert_date'] = pd.to_datetime(df_high['Renewal_Due_Date'],format='mixed') + pd.DateOffset(days=random.choice([1, 4, 8, 10, 3, 5, 7, 8]))
    df_high['alert_tat'] = [random.randint(1, 60) for _ in range(len(df_high))]
    df_high['Date_action_taken'] = pd.to_datetime(df_high['alert_date']) + pd.to_timedelta(df_high['alert_tat'], unit='D')
    open_prob = 0.7
    df_high['alert_status'] = ['open' if random.random() < open_prob else 'Closed' for _ in range(len(df_high))]
    df_high['alert_assgined_to'] = [random.randint(1, 20) for _ in range(len(df_high))]
    df_high['alert_resolved_by'] = [random.randint(1, 20) for _ in range(len(df_high))]
    df_high['account_type'] = 0
    df_high['industry_type'] = 0
    df_high = df_high.drop(['Renewal_Due_Date'], axis=1)
    df_high = df_high.drop_duplicates()
    df_write = pd.concat([df_write, df_high])

    # Medium alert
    filtered_values_med = df[(df[i] <= operationa_dict[i]['med'][0]) & (df[i] >= operationa_dict[i]['med'][1])]
    df_med = filtered_values_med[['cust_id', 'Renewal_Due_Date']].copy()
    df_med['alert_impact'] = 2
    df_med['alert_type'] = 1
    df_med['alert_attribute'] = operationa_dict[i]['attr']
    df_med['alert_date'] = pd.to_datetime(df_med['Renewal_Due_Date'],format='mixed') + pd.DateOffset(days=random.choice([1, 4, 8, 10, 3, 5, 7, 8]))
    df_med['alert_tat'] = [random.randint(1, 60) for _ in range(len(df_med))]
    df_med['Date_action_taken'] = pd.to_datetime(df_med['alert_date']) + pd.to_timedelta(df_med['alert_tat'], unit='D')
    open_prob = 0.7
    df_med['alert_status'] = ['open' if random.random() < open_prob else 'Closed' for _ in range(len(df_med))]
    df_med['alert_assgined_to'] = [random.randint(1, 20) for _ in range(len(df_med))]
    df_med['alert_resolved_by'] = [random.randint(1, 20) for _ in range(len(df_med))]
    df_med['account_type'] = 0
    df_med['industry_type'] = 0
    df_med = df_med.drop(['Renewal_Due_Date'], axis=1)
    df_med = df_med.drop_duplicates()
    df_write = pd.concat([df_write, df_med])

    # Low alert
    filtered_values_low = df[(df[i] < operationa_dict[i]['low'][0]) & (df[i] >= operationa_dict[i]['low'][1])]
    df_low = filtered_values_low[['cust_id', 'Renewal_Due_Date']].copy()
    df_low['alert_impact'] = 1
    df_low['alert_type'] = 1
    df_low['alert_attribute'] = operationa_dict[i]['attr']
    df_low['alert_date'] = pd.to_datetime(df_low['Renewal_Due_Date'],format='mixed') + pd.DateOffset(days=random.choice([1, 4, 8, 10, 3, 5, 7, 8]))
    df_low['alert_tat'] = [random.randint(1, 60) for _ in range(len(df_low))]
    df_low['Date_action_taken'] = pd.to_datetime(df_low['alert_date']) + pd.to_timedelta(df_low['alert_tat'], unit='D')
    open_prob = 0.7
    df_low['alert_status'] = ['open' if random.random() < open_prob else 'Closed' for _ in range(len(df_low))]
    df_low['alert_assgined_to'] = [random.randint(1, 20) for _ in range(len(df_low))]
    df_low['alert_resolved_by'] = [random.randint(1, 20) for _ in range(len(df_low))]
    df_low['account_type'] = 0
    df_low['industry_type'] = 0
    df_low = df_low.drop(['Renewal_Due_Date'], axis=1)
    df_low = df_low.drop_duplicates()
    df_write = pd.concat([df_write, df_low])

StatementMeta(, , , Waiting, )

In [ ]:
###### for already classified values
names_values={
                'Security_Value_Stock':13,
                'Security_Value_Receivables':14,
                'Collateral_Value':15
             }

for i in list(names_values.keys()):
    df_high = df[['cust_id',i,'Renewal_Due_Date']].copy()
    df_high.rename(columns = {i:'alert_impact'}, inplace = True)
    df_high['alert_impact'] = df_high['alert_impact'].map({'High':3,'Medium':2,'Low':1})
    df_high['alert_type']=1
    df_high['alert_attribute']=names_values[i]
    df_high['alert_date']=pd.to_datetime(df_high['Renewal_Due_Date'],format='mixed')+pd.DateOffset(days=random.choice([1, 4, 8, 10, 3,5,7,8]))
    df_high['alert_tat']= [random.randint(1,60) for _ in range(len(df_high))]
    df_high['Date_action_taken']=pd.to_datetime(df_high['alert_date'])+pd.to_timedelta(df_high['alert_tat'],unit='D' )
    open_prob=0.7
    df_high['alert_status']=['open' if random.random()<open_prob else 'Closed' for _ in range(len(df_high))]
    df_high=df_high.drop(['Renewal_Due_Date'], axis=1)
    df_high['alert_assgined_to']= [random.randint(1,20) for _ in range(len(df_high))]
    df_high['alert_resolved_by']= [random.randint(1,20) for _ in range(len(df_high))]
    df_high['account_type']=0
    df_high['industry_type']= 0
    df_high = df_high.drop_duplicates()
    df_write = pd.concat([df_write,df_high])

######## for string 
filtered_values = df.loc[df['Insurance_Alert']=='Yes']
df_high = filtered_values[['cust_id','Renewal_Due_Date']].copy()
df_high['alert_impact']=3
df_high['alert_type']=1
df_high['alert_attribute']=9
df_high['alert_date']=pd.to_datetime(df_high['Renewal_Due_Date'],format='mixed')+pd.DateOffset(days=random.choice([1, 4, 8, 10, 3,5,7,8]))
df_high['alert_tat']= [random.randint(1,60) for _ in range(len(df_high))]
df_high['Date_action_taken']=pd.to_datetime(df_high['alert_date'])+pd.to_timedelta(df_high['alert_tat'],unit='D' )
open_prob=0.7
df_high['alert_status']=['open' if random.random()<open_prob else 'Closed' for _ in range(len(df_high))]
df_high=df_high.drop(['Renewal_Due_Date'], axis=1)
df_high['alert_assgined_to']= [random.randint(1,20) for _ in range(len(df_high))]
df_high['alert_resolved_by']= [random.randint(1,20) for _ in range(len(df_high))]
df_high['account_type']=0
df_high['industry_type']= 0
df_high = df_high.drop_duplicates()

# df_high.to_csv('alert_file.csv',mode='a')
df_write = pd.concat([df_write,df_high])

# df_write=add_internal_values(df_write)
# df_write=add_external_values(df_write)


StatementMeta(, , , Waiting, )

In [ ]:
df_write=spark.createDataFrame(df_write)
df_write.write.format("delta").mode("overwrite").save("Tables/alert_file_1")

StatementMeta(, , , Waiting, )